In [1]:
import os
import re
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
glob.glob(f'../datasets/*.csv')

['../datasets/sub02_03_04_03.csv',
 '../datasets/sub02_03_04_02.csv',
 '../datasets/sub02_03_03_08.csv',
 '../datasets/sub02_03_04_01.csv',
 '../datasets/sub02_03_04_04.csv',
 '../datasets/datasets.csv',
 '../datasets/sub02_03_02_01.csv',
 '../datasets/sub0501.csv',
 '../datasets/sub02_03_02_02.csv',
 '../datasets/sub02_03_02_03.csv',
 '../datasets/sub02_05_02_01.csv',
 '../datasets/sub06_03_02_전처리.csv',
 '../datasets/sub02_05_02_03.csv',
 '../datasets/sub02_03_02_04.csv',
 '../datasets/sub02_05_02_02.csv',
 '../datasets/integrated_datasets.csv',
 '../datasets/librarynotice.csv',
 '../datasets/sub06_03_02.csv',
 '../datasets/sub06_01_01_01.csv',
 '../datasets/sub06_01_01_03.csv',
 '../datasets/sub02_05_01_01.csv',
 '../datasets/sub02_03_03_03.csv',
 '../datasets/sub02_03_03_02.csv',
 '../datasets/sub02_05_01_02.csv',
 '../datasets/sub02_03_03_01.csv',
 '../datasets/sub02_05_01_03.csv',
 '../datasets/sub02_03_03_05.csv',
 '../datasets/sub02_03_03_04.csv',
 '../datasets/sub02_03_03_0

## 정보 통합

In [3]:
def static_info_integeration(content):
    datasets_dir = glob.glob(f'../datasets/{content["datasets_name"]}*.csv')
    datasets = []
    for idx,value in enumerate(datasets_dir):
        file_name = value.split('/')[2].split('.')[0]
        df = pd.read_csv(value)
        df['category'] = [content['category'][idx]] * len(df)
        df['url'] = f'https://www.kumoh.ac.kr/ko/{file_name}.do'
        datasets.append(df.copy())
    df = pd.DataFrame(columns=datasets[0].columns)
    
    for data in datasets:
        df = pd.concat([df, data])
    df.reset_index(inplace = True, drop=True)
    df['source'] = [content['source']]*len(df)
    
    return df


In [4]:
contents = []

# FAQ
contents.append({
    'datasets_name' : 'sub02_03_02',
    'source' : '학사 FAQ',
    'category' : ["학적 및 휴복학", "수강신청", "학점인정", "장학금 및 등록금"]
    }
)

# 학사 안내
contents.append({
    'datasets_name' : 'sub02_03_03',
    'source' : '학사안내',
    'category' :  ["전화번호", "학적", "수강지도상담", "수강신청", "수업 및 성적", "졸업", "증명서", "등록"]
    }
)

# 교육과정
contents.append({
    'datasets_name' : 'sub02_03_04',
    'source' : '교육과정 안내',
    'category' :  ['교육과정구성','이수기준','편성원','교과목의 구분']
    }
)

# 장학지원
contents.append({
    'datasets_name' : 'sub02_05_01',
    'source' : '장학지원',
    'category' :  ['장학금','학자금대출','장학금현황']
    }
)

# 병무/학군단/예비군
contents.append({
    'datasets_name' : 'sub02_05_02',
    'source' : '병무/학군단/예비군',
    'category' :  ['병무','학군단','예비군']
    }
)

In [5]:
df_info = None
for content in contents:
    if df_info is None:
        df_info = static_info_integeration(content)
    else:
        df_info = pd.concat([df_info, static_info_integeration(content)])

In [6]:
df_info.rename(columns={"question":"title", "answer":"content"}, inplace = True)

In [7]:
df_info

,title,content,category,url,source
0,"휴학, 복학, 자퇴등 서식은 어디있나요 ?",교육/대학학사안내/자료실 에 있습니다.,학적 및 휴복학,https://www.kumoh.ac.kr/ko/sub02_03_02_01.do,학사 FAQ
1,★ 증명서 발급 방법 안내,증명서발급 무료 서비스 실시 안내 \n\t\t \n\n『 국립학교의 각종 증명 발급...,학적 및 휴복학,https://www.kumoh.ac.kr/ko/sub02_03_02_01.do,학사 FAQ
2,등록금을 내고 휴학중이였는데 자퇴를 하게 되면 등록금 반환 금액은 어떻게 되나요?,제적시 등록금 반환금액은 다음과 같습니다 . \n\n\t1. 관련 : 대학 등록금에...,학적 및 휴복학,https://www.kumoh.ac.kr/ko/sub02_03_02_01.do,학사 FAQ
3,군제대 날짜가 개학이 좀 지나서인데 복학 할 수 있나요?,군제대 일자가 개학 이후이나 휴가 등으로 학기중 출석에 문제가 되지 않을 경우 복학...,학적 및 휴복학,https://www.kumoh.ac.kr/ko/sub02_03_02_01.do,학사 FAQ
4,복학 할 시기가 남아 있는데 시기를 앞당겨 복학이 가능한가요?,"앞당겨 복학 가능합니다.\n단, 미리 복학하여 역학기 복학 할 경우 수강신청에 주의...",학적 및 휴복학,https://www.kumoh.ac.kr/ko/sub02_03_02_01.do,학사 FAQ
...,...,...,...,...,...
2,학자금대출,학자금대출 \n학자금대출(교내 ☎7065) / 장학재단 콜센터 1599-2000 \...,학자금대출,https://www.kumoh.ac.kr/ko/sub02_05_01_02.do,장학지원
3,장학금 지급 현황,장학금 지급 현황 \n\n\t\t2022학년도 장학금 지급현황 \n\n\t\t(단위...,장학금현황,https://www.kumoh.ac.kr/ko/sub02_05_01_03.do,장학지원
0,병무업무 안내,병무업무 안내 \n재학생 입영연기 \n\n재학생 입영연기자 \n재학생 입영연기자란 ...,병무,https://www.kumoh.ac.kr/ko/sub02_05_02_01.do,병무/학군단/예비군
1,예비군편성 및 훈련안내,예비군편성 및 훈련안내 \n대학직장예비군 편성신고 및 절차 \n\n일 시 : 학교(...,학군단,https://www.kumoh.ac.kr/ko/sub02_05_02_03.do,병무/학군단/예비군


## QnA

In [9]:
import re

def date_processing(date):
    return re.sub(r'\.','-',date)   

In [10]:
df_qna = pd.read_csv('../datasets/sub06_03_02_전처리.csv')
df_qna['date'] = df_qna['date'].map(date_processing)
df_qna['source'] = 'Q&A'

df_qna.drop(columns = ['title'], inplace = True)
df_qna.rename(columns={"question":"title", "answer":"content"}, inplace = True)

In [11]:
df_info.reset_index(drop=True,inplace=True)
df_qna.reset_index(drop=True,inplace=True)

## 정적 정보 통합

In [12]:
df_static_info = pd.concat([df_info, df_qna], axis=0)

## Notice

In [13]:
df_school_notice = pd.read_csv('../datasets/sub06_01_01_01.csv')
df_general_notice = pd.read_csv('../datasets/sub06_01_01_03.csv')
df_computer_notice = pd.read_csv('../datasets/sub0501.csv')
df_library = pd.read_csv('../datasets/librarynotice.csv')

df_school_notice['source'] = '학사공지'
df_general_notice['source'] = '일반공지'
df_computer_notice['source'] = '학과공지'
df_library['source'] = '도서관공지'
df_library['date'] = df_library['date'].map(date_processing)

In [14]:
df_all_dataset = pd.concat([df_static_info, df_school_notice, df_general_notice, df_computer_notice, df_library])

In [15]:
df_all_dataset

,title,content,category,url,source,date
0,"휴학, 복학, 자퇴등 서식은 어디있나요 ?",교육/대학학사안내/자료실 에 있습니다.,학적 및 휴복학,https://www.kumoh.ac.kr/ko/sub02_03_02_01.do,학사 FAQ,NaN
1,★ 증명서 발급 방법 안내,증명서발급 무료 서비스 실시 안내 \n\t\t \n\n『 국립학교의 각종 증명 발급...,학적 및 휴복학,https://www.kumoh.ac.kr/ko/sub02_03_02_01.do,학사 FAQ,NaN
2,등록금을 내고 휴학중이였는데 자퇴를 하게 되면 등록금 반환 금액은 어떻게 되나요?,제적시 등록금 반환금액은 다음과 같습니다 . \n\n\t1. 관련 : 대학 등록금에...,학적 및 휴복학,https://www.kumoh.ac.kr/ko/sub02_03_02_01.do,학사 FAQ,NaN
3,군제대 날짜가 개학이 좀 지나서인데 복학 할 수 있나요?,군제대 일자가 개학 이후이나 휴가 등으로 학기중 출석에 문제가 되지 않을 경우 복학...,학적 및 휴복학,https://www.kumoh.ac.kr/ko/sub02_03_02_01.do,학사 FAQ,NaN
4,복학 할 시기가 남아 있는데 시기를 앞당겨 복학이 가능한가요?,"앞당겨 복학 가능합니다.\n단, 미리 복학하여 역학기 복학 할 경우 수강신청에 주의...",학적 및 휴복학,https://www.kumoh.ac.kr/ko/sub02_03_02_01.do,학사 FAQ,NaN
...,...,...,...,...,...,...
804,DBpia 신규 간행물 안내_2013.8.,NaN,NaN,NaN,도서관공지,2013-08-28
805,2013 전자저널 및 WEB DB 이용안내,NaN,NaN,NaN,도서관공지,2013-07-29
806,Taylor & Francis 오픈엑세스 저널 및 무료 아티클 행사,Taylor & Francis 출판사의 오픈엑세스 저널 및 무료 아티클 행사를 소개...,NaN,NaN,도서관공지,2013-06-28
807,2013년_KERIS_해외전자정보_무료_trial_서비스_안내,2014년 구독 예정인 KERIS 대학 라이선스 및 공동구매 컨소시엄 참여예정 DB...,NaN,NaN,도서관공지,2013-06-25


In [16]:
df_all_dataset.to_csv('../datasets/integrated_datasets.csv', encoding = 'utf8', index = False)